# Sem semaforo


In [16]:
import threading
import random
import time

class ContaSemProtecao:
    def __init__(self, saldo_inicial=1000):
        self.saldo = saldo_inicial
        self.total_sacado = 0

    def consultar_saldo(self):
        return self.saldo

    def sacar(self, valor):
        if self.saldo >= valor:
            print("Thread", threading.current_thread().name, ": Verificou saldo de R$", self.saldo, "para saque de R$", valor)
            saldo_atual = self.saldo
            time.sleep(0.05)
            self.saldo = saldo_atual - valor
            self.total_sacado += valor
            print("Thread", threading.current_thread().name, ": Saque de R$", valor, "realizado. Novo saldo: R$", self.saldo)
            return True
        else:
            print("Thread", threading.current_thread().name, ": Tentou sacar R$", valor, "mas saldo insuficiente: R$", self.saldo)
            return False

def executar_saque_concorrente(conta, num_saques):
    for _ in range(num_saques):
        valor = random.randint(300, 500)
        conta.sacar(valor)

def demonstrar_sobrecarga():
    conta = ContaSemProtecao(saldo_inicial=1000)
    print("Saldo inicial da conta: R$", conta.consultar_saldo())

    threads = []
    num_threads = 10
    saques_por_thread = 3

    for i in range(num_threads):
        t = threading.Thread(target=executar_saque_concorrente, args=(conta, saques_por_thread), name="Cliente-" + str(i+1))
        threads.append(t)

    for t in threads:
        t.start()

    for t in threads:
        t.join()

    print("============================================================")
    print("⚠️  RESULTADO DA SIMULAÇÃO:")
    print("Saldo inicial: R$", 1000)
    print("Total sacado: R$", conta.total_sacado)
    print("Saldo final:", "\033[91mR$" + str(conta.consultar_saldo()) + "\033[0m")

    if conta.saldo < 0:
        print("❗ RACE CONDITION DETECTADA: Saldo ficou NEGATIVO! Exemplo de erro grave em sistemas concorrentes.")
    elif conta.total_sacado > 1000:
        print("❗ RACE CONDITION DETECTADA: Foram sacados R$", conta.total_sacado, "de uma conta com apenas R$1000!")
    else:
        print("✅ Nenhuma inconsistência detectada.")
    print("============================================================")

if __name__ == "__main__":
    print("============================================================")
    demonstrar_sobrecarga()


Saldo inicial da conta: R$ 1000
Thread Cliente-1 : Verificou saldo de R$ 1000 para saque de R$ 361
Thread Cliente-2 : Verificou saldo de R$ 1000 para saque de R$ 391
Thread Cliente-3 : Verificou saldo de R$ 1000 para saque de R$ 347
Thread Cliente-4 : Verificou saldo de R$ 1000 para saque de R$ 456
Thread Cliente-5 : Verificou saldo de R$ 1000 para saque de R$ 346
Thread Cliente-6 : Verificou saldo de R$ 1000 para saque de R$ 440
Thread Cliente-7 : Verificou saldo de R$ 1000 para saque de R$ 394
Thread Cliente-8 : Verificou saldo de R$ 1000 para saque de R$ 336
Thread Cliente-9 : Verificou saldo de R$ 1000 para saque de R$ 304
Thread Cliente-10 : Verificou saldo de R$ 1000 para saque de R$ 337
Thread Cliente-1 : Saque de R$ 361 realizado. Novo saldo: R$ 639
Thread Cliente-1 : Verificou saldo de R$ 639 para saque de R$ 427
Thread Cliente-2 : Saque de R$ 391 realizado. Novo saldo: R$ 609
Thread Cliente-2 : Verificou saldo de R$ 609 para saque de R$ 429
Thread Cliente-4 : Saque de R$ 456 

# Com semaforo


In [17]:
import threading
import random
import time

class ContaComSemaforo:
    def __init__(self, saldo_inicial=1000):
        self.saldo = saldo_inicial
        self.total_sacado = 0
        self.semaforo = threading.Semaphore()  # Semáforo binário (mutex)

    def consultar_saldo(self):
        return self.saldo

    def sacar(self, valor):
        self.semaforo.acquire()  # Início da seção crítica
        try:
            if self.saldo >= valor:
                print("Thread", threading.current_thread().name, ": Verificou saldo de R$", self.saldo, "para saque de R$", valor)
                saldo_atual = self.saldo
                time.sleep(0.05)
                self.saldo = saldo_atual - valor
                self.total_sacado += valor
                print("Thread", threading.current_thread().name, ": Saque de R$", valor, "realizado. Novo saldo: R$", self.saldo)
                return True
            else:
                print("Thread", threading.current_thread().name, ": Tentou sacar R$", valor, "mas saldo insuficiente: R$", self.saldo)
                return False
        finally:
            self.semaforo.release()  # Fim da seção crítica

def executar_saque_concorrente(conta, num_saques):
    for _ in range(num_saques):
        valor = random.randint(300, 500)
        conta.sacar(valor)

def demonstrar_execucao_segura():
    conta = ContaComSemaforo(saldo_inicial=1000)
    print("Saldo inicial da conta: R$", conta.consultar_saldo())

    threads = []
    num_threads = 10
    saques_por_thread = 3

    for i in range(num_threads):
        t = threading.Thread(target=executar_saque_concorrente, args=(conta, saques_por_thread), name="Cliente-" + str(i+1))
        threads.append(t)

    for t in threads:
        t.start()

    for t in threads:
        t.join()

    print("============================================================")
    print("✅ RESULTADO DA SIMULAÇÃO COM SEMÁFORO (SINCRONIZADO):")
    print("Saldo inicial: R$", 1000)
    print("Total sacado: R$", conta.total_sacado)

    # Exibindo o saldo final em verde
    print("\033[32mSaldo final: R$", conta.consultar_saldo(), "\033[0m")

    if conta.saldo < 0:
        print("❌ ERRO: Saldo ficou NEGATIVO! Mesmo com semáforo, algo está errado.")
    elif conta.total_sacado > 1000:
        print("❌ ERRO: Foram sacados R$", conta.total_sacado, "de uma conta com apenas R$1000!")
    else:
        print("✅ Execução segura")
    print("============================================================")

if __name__ == "__main__":

    demonstrar_execucao_segura()


Saldo inicial da conta: R$ 1000
Thread Cliente-1 : Verificou saldo de R$ 1000 para saque de R$ 485
Thread Cliente-1 : Saque de R$ 485 realizado. Novo saldo: R$ 515
Thread Cliente-1 : Verificou saldo de R$ 515 para saque de R$ 391
Thread Cliente-1 : Saque de R$ 391 realizado. Novo saldo: R$ 124
Thread Cliente-1 : Tentou sacar R$ 340 mas saldo insuficiente: R$ 124
Thread Cliente-3 : Tentou sacar R$ 309 mas saldo insuficiente: R$ 124
Thread Cliente-3 : Tentou sacar R$ 340 mas saldo insuficiente: R$ 124
Thread Cliente-3 : Tentou sacar R$ 443 mas saldo insuficiente: R$ 124
Thread Cliente-4 : Tentou sacar R$ 392 mas saldo insuficiente: R$ 124
Thread Cliente-4 : Tentou sacar R$ 348 mas saldo insuficiente: R$ 124
Thread Cliente-4 : Tentou sacar R$ 430 mas saldo insuficiente: R$ 124
Thread Cliente-6 : Tentou sacar R$ 365 mas saldo insuficiente: R$ 124
Thread Cliente-6 : Tentou sacar R$ 495 mas saldo insuficiente: R$ 124
Thread Cliente-6 : Tentou sacar R$ 353 mas saldo insuficiente: R$ 124
Threa